In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/ChartVQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answers,question_id,most_likely,responses
0,[/root/autodl-tmp/ChartVQA/train/png/two_col_1...,[What was the unadjusted unemployment rate of ...,[5.2],[two_col_1435.png],{'embedding': [[[-0.00982666 0.00026894 0.00...,None
1,[/root/autodl-tmp/ChartVQA/train/png/two_col_2...,[What was the government expenditure in the fi...,[5.49],[two_col_22964.png],{'embedding': [[[ 0.00328064 0.00071335 0.00...,None
2,[/root/autodl-tmp/ChartVQA/train/png/two_col_1...,"[What was the rate of C-sections per 1,000 liv...",[548.9],[two_col_1565.png],{'embedding': [[[-0.00982666 0.00026894 0.00...,None
3,[/root/autodl-tmp/ChartVQA/train/png/multi_col...,[How many of Qdoba's restaurants were company ...,[385],[multi_col_1089.png],{'embedding': [[[-0.01599121 -0.01062012 -0.01...,None
4,[/root/autodl-tmp/ChartVQA/train/png/two_col_4...,[What was the revenue of General Dynamics from...,[26303],[two_col_40932.png],{'embedding': [[[ 0.00020885 -0.00279236 0.00...,None


In [5]:
i=111
df.iloc[i]['most_likely']['response']

['15.02']

In [6]:
df.iloc[i]['question']

['How many people in the EU were unemployed in June 2020?\nAnswer the question using a single word or digit number.\n']

In [7]:
df.iloc[i]['answers']

['15.02']

In [8]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [9]:
embeddings=np.concatenate(embeddings,axis=0)

In [10]:
embeddings.shape

(312, 29, 3584)

In [11]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [12]:
gt_label.shape

(312,)

In [13]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [14]:
from answer_judge import load_judge

In [15]:
is_correct=[]
judge=load_judge('ChartQA')
for i,x in enumerate(resps):
    if judge.check(x,gt_label[i]):
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [16]:
is_correct.shape

(312,)

In [17]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 312 responses. 265 answer correctly and 47 answer wrongly


In [19]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:0
validset length:249
testset length:63
trainset postive: 0 negative: 0
valset postive: 212 negative: 37
testset postive: 53 negative: 10


In [20]:
all_embeddings = embeddings
all_embeddings.shape

(312, 29, 3584)

In [21]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [24]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCAKernel,PCALinear

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [25]:
def evaluator(n_components,i_layer,X,y):
    kpca=PCALinear(embedding_train[:,i_layer,:],n_components)
    scores=kpca.get_score(X[:,i_layer,:])
    return roc_auc_score(y,scores)

In [26]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

ValueError: Found array with 0 sample(s) (shape=(0, 3584)) while a minimum of 1 is required by PCA.

In [24]:
best_paras

{'n_components': 2, 'layer': 26, 'best_score': np.float64(0.666986301369863)}

In [25]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [26]:
discriminator=PCADiscriminator(embedding_train[:,best_layer,:],best_paras['n_components'],kernel='linear')
scores=discriminator.get_score2(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [27]:
scores=discriminator.get_score2(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.5666666666666667 AUROC:0.6280978049366156


In [28]:
scores=discriminator.get_score2(embedding_test[:,best_layer,:])
y_test=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,y_test)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.57847533632287 AUROC:0.6631551293839484


Train Linear Prob to Get Better Results

In [29]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [30]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [31]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.5762331838565022 AUROC:0.6481544726126363
